# Import Libraries

In [190]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

# Import prepared dataframe with matches for the last 3 seasons of EPL

In [236]:
matches = pd.read_csv("matches.csv", index_col=0)

In [237]:
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,15.0,1.0,0.0,0.0,2023,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,12.6,0.0,0.0,0.0,2023,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.6,0.0,0.0,0.0,2023,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,23.0,8.0,16.6,1.0,0.0,0.0,2023,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Match Report,NaN,22.0,8.0,15.3,1.0,0.0,0.0,2023,Arsenal


In [238]:
matches.shape

(1674, 27)

In [239]:
# For the convenience we make all the column-names with lowercase and replace dashes with spaces
matches.columns = [c.lower() for c in matches.columns]
matches["team"] = matches["team"].str.replace("-", " ")
matches["team"].value_counts()

Arsenal                     84
Manchester City             84
Leicester City              84
Wolverhampton Wanderers     84
Southampton                 84
West Ham United             84
Aston Villa                 84
Everton                     84
Newcastle United            84
Tottenham Hotspur           84
Liverpool                   83
Leeds United                83
Manchester United           83
Chelsea                     83
Crystal Palace              83
Brighton and Hove Albion    83
Burnley                     76
Brentford                   46
Fulham                      46
Norwich City                38
West Bromwich Albion        38
Sheffield United            38
Watford                     38
Nottingham Forest            8
Bournemouth                  8
Name: team, dtype: int64

In [240]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

In [241]:
# Delete some unnecessary columns
del matches["comp"]
del matches["notes"]

In [242]:
# Change type of the date-column into date-type
matches["date"] = pd.to_datetime(matches["date"])

In [243]:
# Create new column 'Tareget' with values 1 (case win) and 0 (case draw or lose)
matches["target"] = (matches["result"] == "W").astype("int")

In [244]:
matches

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,match report,sh,sot,dist,fk,pk,pkatt,season,team,target
0,2022-08-05,20:00,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,1.0,...,Match Report,10.0,2.0,15.0,1.0,0.0,0.0,2023,Arsenal,1
1,2022-08-13,15:00,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,2.6,...,Match Report,19.0,7.0,12.6,0.0,0.0,0.0,2023,Arsenal,1
2,2022-08-20,17:30,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,1.8,...,Match Report,14.0,6.0,14.6,0.0,0.0,0.0,2023,Arsenal,1
3,2022-08-27,17:30,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,2.4,...,Match Report,23.0,8.0,16.6,1.0,0.0,0.0,2023,Arsenal,1
4,2022-08-31,19:30,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,2.7,...,Match Report,22.0,8.0,15.3,1.0,0.0,0.0,2023,Arsenal,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Matchweek 34,Sun,Away,L,0.0,4.0,Tottenham,0.5,...,Match Report,8.0,1.0,17.4,0.0,0.0,0.0,2021,Sheffield United,0
39,2021-05-08,15:00,Matchweek 35,Sat,Home,L,0.0,2.0,Crystal Palace,0.7,...,Match Report,7.0,0.0,11.4,1.0,0.0,0.0,2021,Sheffield United,0
40,2021-05-16,19:00,Matchweek 36,Sun,Away,W,1.0,0.0,Everton,1.6,...,Match Report,10.0,3.0,17.0,0.0,0.0,0.0,2021,Sheffield United,1
41,2021-05-19,18:00,Matchweek 37,Wed,Away,L,0.0,1.0,Newcastle Utd,0.8,...,Match Report,11.0,1.0,16.0,1.0,0.0,0.0,2021,Sheffield United,0


In [245]:
# Digitalization of the venue - 1 for Home and 0 - Away
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [246]:
# Digitalization of the opponent (to be not the club name, but just a number
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [247]:
# Making time to int
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [248]:
# Making day to number
matches["day_code"] = matches["date"].dt.dayofweek

In [249]:
matches.head()

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,fk,pk,pkatt,season,team,target,venue_code,opp_code,hour,day_code
0,2022-08-05,20:00,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,1.0,...,1.0,0.0,0.0,2023,Arsenal,1,0,7,20,4
1,2022-08-13,15:00,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,2.6,...,0.0,0.0,0.0,2023,Arsenal,1,1,11,15,5
2,2022-08-20,17:30,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,1.8,...,0.0,0.0,0.0,2023,Arsenal,1,0,2,17,5
3,2022-08-27,17:30,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,2.4,...,1.0,0.0,0.0,2023,Arsenal,1,1,9,17,5
4,2022-08-31,19:30,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,2.7,...,1.0,0.0,0.0,2023,Arsenal,1,1,1,19,2


In [250]:
# Let's create an object of the RandomForestClassifier class to use then for prediction
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
# n_estimators– number of trees the algorithm builds before averaging the predictions
# min_samples_split - The minimum number of samples required to split an internal node
# random_state - Controls the randomness of the estimator.

In [251]:
# Creation of the train data
train = matches[matches["date"] < '2022-07-01']

In [252]:
# Creation of the test data (to check the result of the test, to compare)
test = matches[matches["date"] > '2022-07-01']

In [253]:
# Definition of the metrics that are going to use for training
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [254]:
# Training
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [255]:
# Prediction based on predictors
preds = rf.predict(test[predictors])
preds

array([1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [256]:
# Error is a metric that let to check accuracy of the prediction - as it higher than better
error = accuracy_score(test["target"], preds)
error

0.6493506493506493

In [257]:
# Let's create dataframe that shows actual result with predicted.
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))
combined

,actual,predicted
0,1,1
1,1,0
2,1,0
3,1,1
4,1,0
...,...,...
4,0,0
5,0,0
6,0,0
7,0,0


In [258]:
# Let's check where algorith was right, and where it was wrong
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,79,20
1,34,21


In [259]:
# Above metrics mean that the model done following results:
# 79 - True negative - it was predicted 0-result, where it should be actualy 0-result
# 34 - False negative - it was predicted 0-result, where it should be 1-result
# 20 - False positive - it was predicted 1-result where it should be 0-result
#  21 - True positive - it was predicted 1-result where it should be 1-result

In [260]:
confusion_matrix(combined["actual"].values, combined["predicted"])

array([[79, 20],
       [34, 21]], dtype=int64)

In [261]:
# Next - let's check precision_score - is the fraction of predicted positives events 
# that are actually positive
precision_score(test["target"], preds)

0.5121951219512195

In [262]:
# To increase prediction accurace we'll add new metrics
# rolling metrics for the last 3 games before match
# First thing - we make group by team our table 
grouped_matches = matches.groupby("team")

In [263]:
# Just to check how the group is look like
group = grouped_matches.get_group("Manchester City").sort_values("date")

In [264]:
# Now we want to define a function that counts metrics for the last 3 games before
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [265]:
# Here we assign a columns that we'd like to count rolling data
cols = ["gf", "ga", "xg", "xga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,gf_rolling,ga_rolling,xg_rolling,xga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
5,2020-10-17,17:30,Matchweek 5,Sat,Home,W,1.0,0.0,Arsenal,1.5,...,2.000000,2.333333,1.433333,1.766667,17.333333,4.666667,18.900000,1.333333,0.333333,0.333333
7,2020-10-24,12:30,Matchweek 6,Sat,Away,D,1.0,1.0,West Ham,1.1,...,1.333333,2.000000,1.300000,1.733333,17.333333,3.666667,17.733333,0.666667,0.000000,0.000000
9,2020-10-31,12:30,Matchweek 7,Sat,Away,W,1.0,0.0,Sheffield Utd,1.5,...,1.000000,0.666667,1.366667,1.033333,16.666667,4.333333,18.233333,0.666667,0.000000,0.000000
11,2020-11-08,16:30,Matchweek 8,Sun,Home,D,1.0,1.0,Liverpool,1.6,...,1.000000,0.333333,1.366667,0.633333,14.333333,6.666667,18.466667,1.000000,0.000000,0.000000
12,2020-11-21,17:30,Matchweek 9,Sat,Away,L,0.0,2.0,Tottenham,1.3,...,1.000000,0.666667,1.400000,0.800000,12.000000,5.666667,19.366667,1.000000,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-08-27,15:00,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,2.5,...,3.000000,1.000000,2.200000,0.766667,18.333333,6.000000,16.300000,0.666667,0.333333,0.333333
5,2022-08-31,19:30,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,3.7,...,3.666667,1.666667,2.266667,0.666667,20.000000,7.333333,14.600000,0.333333,0.000000,0.000000
6,2022-09-03,17:30,Matchweek 6,Sat,Away,D,1.0,1.0,Aston Villa,1.4,...,4.333333,1.666667,2.966667,0.766667,19.000000,8.000000,14.100000,0.333333,0.000000,0.000000
9,2022-09-17,12:30,Matchweek 8,Sat,Away,W,3.0,0.0,Wolves,1.4,...,3.666667,1.000000,2.533333,0.233333,16.000000,6.000000,14.300000,0.333333,0.000000,0.000000


In [266]:
matches_rolling.columns

Index(['date', 'time', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt',
       'season', 'team', 'target', 'venue_code', 'opp_code', 'hour',
       'day_code', 'gf_rolling', 'ga_rolling', 'xg_rolling', 'xga_rolling',
       'sh_rolling', 'sot_rolling', 'dist_rolling', 'fk_rolling', 'pk_rolling',
       'pkatt_rolling'],
      dtype='object')

In [267]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [268]:
matches_rolling

date   time        round  day venue result  \
team                                                                          
Arsenal                 6  2020-10-04  14:00  Matchweek 4  Sun  Home      W   
                        7  2020-10-17  17:30  Matchweek 5  Sat  Away      L   
                        9  2020-10-25  19:15  Matchweek 6  Sun  Home      L   
                        11 2020-11-01  16:30  Matchweek 7  Sun  Away      W   
                        13 2020-11-08  19:15  Matchweek 8  Sun  Home      L   
...                               ...    ...          ...  ...   ...    ...   
Wolverhampton Wanderers 4  2022-08-28  14:00  Matchweek 4  Sun  Home      D   
                        5  2022-08-31  19:30  Matchweek 5  Wed  Away      D   
                        6  2022-09-03  15:00  Matchweek 6  Sat  Home      W   
                        7  2022-09-17  12:30  Matchweek 8  Sat  Home      L   
                        8  2022-10-01  17:30  Matchweek 9  Sat  Away      L   

                             gf   ga         opponent   xg  ...  gf_rolling  \
team                                                        ...               
Arsenal                 6   2.0  1.0    Sheffield Utd  0.4  ...    2.000000   
                        7   0.0  1.0  Manchester City  0.9  ...    1.666667   
                        9   0.0  1.0   Leicester City  0.9  ...    1.000000   
                        11  1.0  0.0   Manchester Utd  1.1  ...    0.666667   
                        13  0.0  3.0      Aston Villa  1.5  ...    0.333333   
...                         ...  ...              ...  ...  ...         ...   
Wolverhampton Wanderers 4   1.0  1.0    Newcastle Utd  0.8  ...    0.333333   
                        5   0.0  0.0      Bournemouth  1.5  ...    0.333333   
                        6   1.0  0.0      Southampton  0.7  ...    0.333333   
                        7   0.0  3.0  Manchester City  0.5  ...    0.666667   
                        8   0.0  2.0         West Ham  0.7  ...    0.333333   

                            ga_rolling  xg_rolling xga_rolling sh_rolling  \
team                                                                        
Arsenal                 6     1.333333    1.466667    1.766667   7.666667   
                        7     1.666667    1.000000    1.766667   5.333333   
                        9     1.666667    0.833333    1.633333   7.000000   
                        11    1.000000    0.733333    0.966667   9.666667   
                        13    0.666667    0.966667    1.000000   9.666667   
...                                ...         ...         ...        ...   
Wolverhampton Wanderers 4     1.000000    0.933333    1.100000  14.000000   
                        5     0.666667    0.766667    1.566667  12.333333   
                        6     0.666667    1.033333    1.333333  15.666667   
                        7     0.333333    1.000000    1.166667  11.333333   
                        8     1.000000    0.900000    0.966667  10.000000   

                           sot_rolling dist_rolling  fk_rolling  pk_rolling  \
team                                                                          
Arsenal                 6     3.666667    14.733333    0.666667    0.000000   
                        7     3.666667    15.766667    0.000000    0.000000   
                        9     3.666667    16.733333    0.666667    0.000000   
                        11    4.000000    16.033333    1.000000    0.000000   
                        13    2.666667    18.033333    1.000000    0.333333   
...                                ...          ...         ...         ...   
Wolverhampton Wanderers 4     3.333333    19.333333    0.666667    0.000000   
                        5     2.666667    19.600000    1.000000    0.000000   
                        6     3.333333    19.066667    1.000000    0.000000   
                        7     3.000000    16.600000    0.666667    0.000000   
                        8  

In [269]:
# Let's remove the first column
matches_rolling = matches_rolling.droplevel('team')

In [270]:
matches_rolling.index = range(matches_rolling.shape[0])

In [271]:
matches_rolling

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,gf_rolling,ga_rolling,xg_rolling,xga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2020-10-04,14:00,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,0.4,...,2.000000,1.333333,1.466667,1.766667,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000
1,2020-10-17,17:30,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,0.9,...,1.666667,1.666667,1.000000,1.766667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000
2,2020-10-25,19:15,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,0.9,...,1.000000,1.666667,0.833333,1.633333,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000
3,2020-11-01,16:30,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,1.1,...,0.666667,1.000000,0.733333,0.966667,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000
4,2020-11-08,19:15,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,1.5,...,0.333333,0.666667,0.966667,1.000000,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,2022-08-28,14:00,Matchweek 4,Sun,Home,D,1.0,1.0,Newcastle Utd,0.8,...,0.333333,1.000000,0.933333,1.100000,14.000000,3.333333,19.333333,0.666667,0.000000,0.000000
1592,2022-08-31,19:30,Matchweek 5,Wed,Away,D,0.0,0.0,Bournemouth,1.5,...,0.333333,0.666667,0.766667,1.566667,12.333333,2.666667,19.600000,1.000000,0.000000,0.000000
1593,2022-09-03,15:00,Matchweek 6,Sat,Home,W,1.0,0.0,Southampton,0.7,...,0.333333,0.666667,1.033333,1.333333,15.666667,3.333333,19.066667,1.000000,0.000000,0.000000
1594,2022-09-17,12:30,Matchweek 8,Sat,Home,L,0.0,3.0,Manchester City,0.5,...,0.666667,0.333333,1.000000,1.166667,11.333333,3.000000,16.600000,0.666667,0.000000,0.000000


In [272]:
# Let's make a function that allows to make all the processes that we done before at
# at the same time
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-07-01']
    test = data[data["date"] > '2022-07-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [273]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)

In [274]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,81,14
1,39,14


In [275]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [276]:
# Creating ot the class which allows to replace some names of the clubs to the similar
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Brighton and Hove Albion": "Brighton", 
    "Manchester United": "Manchester Utd", 
    "Newcastle United": "Newcastle Utd", 
    "Tottenham Hotspur": "Tottenham", 
    "West Ham United": "West Ham", 
    "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)

In [277]:
# Applying of the replacement to the df
combined["new_team"] = combined["team"].map(mapping)

In [278]:
# Merging df with itself - for each team there are one match at home, and another team 
# has the same match away 
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [279]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,1,0,2022-08-05,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
1,1,0,2022-08-13,Arsenal,Leicester City,W,Arsenal,0,0,Leicester City,Arsenal,L,Leicester City
2,1,0,2022-08-27,Arsenal,Fulham,W,Arsenal,0,0,Fulham,Arsenal,L,Fulham
3,1,1,2022-08-31,Arsenal,Aston Villa,W,Arsenal,0,0,Aston Villa,Arsenal,L,Aston Villa
4,0,0,2022-09-04,Arsenal,Manchester Utd,L,Arsenal,1,0,Manchester United,Arsenal,W,Manchester Utd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,0,0,2022-08-28,Wolverhampton Wanderers,Newcastle Utd,D,Wolves,0,0,Newcastle United,Wolves,D,Newcastle Utd
133,0,0,2022-08-31,Wolverhampton Wanderers,Bournemouth,D,Wolves,0,0,Bournemouth,Wolves,D,Bournemouth
134,1,0,2022-09-03,Wolverhampton Wanderers,Southampton,W,Wolves,0,1,Southampton,Wolves,L,Southampton
135,0,0,2022-09-17,Wolverhampton Wanderers,Manchester City,L,Wolves,1,0,Manchester City,Wolves,W,Manchester City


In [282]:
# The last step is to check where it was predicted right for the both teams - one has won
# and another - lose
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()

1    11
0    10
Name: actual_x, dtype: int64